__Support Network Application__

Authors: Sammi Harper & James Shoemaker

Date: 02/01/2025

Summary: This application will allow users to interact with a peers and counselors network. Users can create accounts, send messages to peers and counselors, messagin counselors allows for annonymous messages, and post on the discussion board from other users.
        


In [71]:
from tkinter import *
import tkinter as tk
from tkinter.font import Font
from tkinter import messagebox
from tkinter import PhotoImage
from tkinter import Button
import sqlite3
import re

# opened to view the photo - will delete comment later
#test test
class CounselorChat:
    def __init__(self):
        # Initialize top level window
        self.counselor = Toplevel()
        self.counselor.title(" Resource Treatment Center || Counselor Chat")
        self.counselor.geometry('800x950') # updated deminsions to show feed and post box relative to display size
        self.center_window(800, 950)      # JS centers window
        self.counselor.resizable(True, True) # This should probably be true once the function is defined to center all aspects 
        self.counselor.config(background="light grey")
        
        
        # Create a Canvas widget to hold background image and label/button widgets
        self.canvas = Canvas(self.counselor, width=800, height=950)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="fist-bump.png") 
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        
        
        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)
        
        # define Colors
        self.label_font_color = "navy"
        
        # Label background color
        self.label_bg = "white"

        # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg=("navy")
        self.button_fg=("light gray")
        
        # Diss Board Positng background color
        self.post_background = "light grey"
        

        # title for the Profile  window
        self.welcome_label = Label(self.counselor, text="RTC || Counselor Chat ", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(400, 50, window=self.welcome_label)
        
        self.detailMessage = Label(self.counselor, text="Private message a counselor anytime! ", fg=self.label_font_color, font=self.entry_font,  bg=self.label_bg)
        # position the label on the top of the window centered
        self.canvas.create_window(400, 100, window=self.detailMessage)
        
        # label and dropdown for active peer account holders
        self.peer_list_label = Label(self.counselor, text="Select a Counselor: ", font=self.button_font, bg=self.label_bg, fg=self.label_font_color)
        self.canvas.create_window(100, 150, window=self.peer_list_label) 
        
        
        #TODO: create functionallity for pulling counselor names fro to private message from a table to database and place a dropdown menu using MenuOption-- should use SQLite per Storer -- SH
        
        
        # exit button for window
        exit_button = Button(self.counselor, text="EXIT", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.counselor.destroy)
        self.canvas.create_window(750, 25, window=exit_button)
        
        # Entry Box for message to board
        message_entry_label = Label(self.counselor, text="Write Message:", font=self.button_font, bg=self.label_bg, fg=self.button_bg)
        self.canvas.create_window(100, 600, window=message_entry_label)
        
        # list box to display messages 
        message_feed = Listbox(self.counselor, font=self.entry_font, width=50, height=18, bg=self.post_background)
        self.canvas.create_window(400, 375, window=message_feed)
        
        # Input message box
        message_entry = Entry(self.counselor, font=self.entry_font, width=50, bg=self.post_background)
        self.canvas.create_window(400, 600, window=message_entry)
        
        # button to submit post to board
        post_button = Button(self.counselor, text="SEND", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.counselorMessage)
        self.canvas.create_window(700, 600, window=post_button)
        
    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.counselor.winfo_screenwidth()
        screen_height = self.counselor.winfo_screenheight()
        # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.counselor.geometry(f"{width}x{height}+{x}+{y}")    
            
    
    def pullMessage(self):
        # add method to pull messages from database when called
        pass
    
    def displayMessage(self):
        # add method to display messages from database when called
        pass
    
    def counselorMessage(self):
        # add method to send message to selected counselor from drop down
        pass
        

In [ ]:
class PeerChat:
    
    def __init__(self, db_path="support_network.db"):
        """
        Initializes the PeerChat class with a SQLite database connection and GUI components.
        
        Websites that helped understand how to complete the chat interface:
        https://stackoverflow.com/questions/42062391/how-to-create-a-chat-window-with-tkinter
        https://www.w3resource.com/python-exercises/tkinter/python-tkinter-layout-management-exercise-7.php
        https://www.sqlite.org/index.html
        
        """
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.peer_chat_table()
        self.create_interface()
        
    def create_interface(self):
        """
        Initializes the peer chat GUI.
        """
        # Initialize top level window
        self.peer = Toplevel()
        self.peer.title(" Resource Treatment Center || Peer Chat")
        self.peer.geometry('800x950') # updated deminsions to show feed and post box relative to display size
        self.center_window(800, 950)      # JS centers window
        self.peer.resizable(False, False) # This should probably be true once the function is defined to center all aspects 
        self.peer.config(background="light grey")
        
        
        # Create a Canvas widget to hold background image and label/button widgets
        self.canvas = Canvas(self.peer, width=800, height=950)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="fist-bump.png") 
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        
        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)

	    # define Colors
        self.label_font_color = "navy"

	    # Label background color
        self.label_bg = "white"

	    # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg=("navy")
        self.button_fg=("light gray")

	    # Diss Board Positng background color
        self.post_background = "light grey"
        

        # title for the Profile  window
        self.welcome_label = Label(self.peer, text="RTC || Peer Chat ", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(400, 50, window=self.welcome_label)
        
        self.detailMessage = Label(self.peer, text="Private message a friend anytime! ", fg=self.label_font_color, font=self.entry_font,  bg=self.label_bg)
        # position the label on the top of the window centered
        self.canvas.create_window(400, 100, window=self.detailMessage)
        
        # label and dropdown for active peer account holders
        self.peer_list_label = Label(self.peer, text="Select a Friend: ", font=self.button_font, bg=self.label_bg, fg=self.label_font_color)
        self.canvas.create_window(100, 150, window=self.peer_list_label) 
        
        
        # dropdown menu for selecting a user to message
        self.selected_user = StringVar()
        # default message for dropdown menu
        self.selected_user.set("Select User")
        # get user names from database 
        users = self.get_patient_users()
       # Create dropdown menu with user list
        self.peer_dropdown = OptionMenu(self.peer, self.selected_user, *users)
        # Position dropdown in GUI window
        self.canvas.create_window(300, 150, window=self.peer_dropdown)
        
        
        # exit button for window
        self.exit_button = Button(self.peer, text="EXIT", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.peer.destroy)
        self.canvas.create_window(750, 25, window=self.exit_button)
        
        # Entry Box for message to board
        self.message_entry_label = Label(self.peer, text="Write Message:", font=self.button_font, bg=self.label_bg, fg=self.button_bg)
        self.canvas.create_window(100, 600, window=self.message_entry_label)
        
        # list box to display messages 
        self.message_feed = Listbox(self.peer, font=self.entry_font, width=50, height=18, bg=self.post_background)
        self.canvas.create_window(400, 375, window=self.message_feed)
        
        # Input message box
        self.message_entry = Entry(self.peer, font=self.entry_font, width=50, bg=self.post_background)
        self.canvas.create_window(400, 600, window=self.message_entry)
        
        # button to submit post to board
        self.post_button = Button(self.peer, text="SEND", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.send_peer_message)
        self.canvas.create_window(700, 600, window=self.post_button)
        
        
    def peer_chat_table(self):
        """
        Creates the peer_chat table if it does not exist.
        """
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS{
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                sender_id INTEGER,
                receiver_id INTEGER,
                message TEXT,
                is_read INTEGER DEFAULT 0, -- this defaults the messages read to 0 as a user reads messages it will count up 
                FOREIGN KEY (sender_id) REFERENCES patient_users(p_id),
                FOREIGN KEY (receiver_id) REFERENCES patient_users(p_id)
            )
        ''')
        self.conn.commit()
        
        
    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.peer.winfo_screenwidth()
        screen_height = self.peer.winfo_screenheight()
        # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.peer.geometry(f"{width}x{height}+{x}+{y}")    
        
        
    def pullMessage(self):
        # add method to pull messages from database when called
        pass
    
    def displayMessage(self):
        # add method to display messages from database when called
        pass
    
    def peerMessage(self):
        # add method to send message to selected user from drop down
        pass
        
        
        

In [73]:
class DissBoard: ## -- add layout for disscussion board SH 
    """
    This class is used to create the discussion board window. 
    Last modified by SH 2/13/25
    """
    def __init__(self):
        # Initialize top level window
        self.dissBoard = Toplevel()
        self.dissBoard.title(" Resource Treatment Center || DISSCUSSION BOARD")
        self.dissBoard.geometry('800x950') # updated deminsions to show feed and post box relative to display size
        self.center_window(800, 950)      # JS centers window
        self.dissBoard.resizable(True, True) # This should probably be true once the function is defined to center all aspects -- SH changed to True 
        self.dissBoard.config(background="light grey")
        
        
        # Create a Canvas widget to hold background image and label/button widgets
        self.canvas = Canvas(self.dissBoard, width=800, height=950)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="fist-bump.png") 
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        
        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)

	# define Colors
        self.label_font_color = "navy"

	# Label background color
        self.label_bg = "white"

	# Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg=("navy")
        self.button_fg=("light gray")

	# Diss Board Positng background color
        self.post_background = "light grey"

        # title for the Profile  window
        self.welcome_label = Label(self.dissBoard, text="RTC || Disscussion Board ", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(400, 50, window=self.welcome_label)
        
        self.detailMessage = Label(self.dissBoard, text="Comment, chat, and openly discuss any aspect of your life with our members!", fg=self.label_font_color, font=self.entry_font,  bg=self.label_bg)
        # position the label on the top of the window centered
        self.canvas.create_window(400, 100, window=self.detailMessage)
        
        back_button = Button(self.dissBoard, text="EXIT", font=self.button_font, bg="navy", fg="light gray", command=self.dissBoard.destroy)
        self.canvas.create_window(750, 25, window=back_button)
        
        # Message feed window to display messages from other users
        post_feed = Listbox(self.dissBoard, font=self.entry_font, width=50, height=35, bg=self.post_background)
        self.canvas.create_window(400, 475, window=post_feed)
        
        # Entry Box for posting to board
        post_entry_label = Label(self.dissBoard, text="Write Post:", font=self.button_font, bg=self.label_bg, fg=self.button_bg)
        self.canvas.create_window(75, 850, window=post_entry_label)
        post_entry = Entry(self.dissBoard, font=self.entry_font, width=50, bg=self.post_background)
        self.canvas.create_window(400, 850, window=post_entry)
        
        # button to submit post to board
        post_button = Button(self.dissBoard, text="SEND", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.boardFeed)
        self.canvas.create_window(700, 850, window=post_button)
        
    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.dissBoard.winfo_screenwidth()
        screen_height = self.dissBoard.winfo_screenheight()   
        # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.dissBoard.geometry(f"{width}x{height}+{x}+{y}") 
        
        
    def boardFeed():
        
        """ Function for submitting to dissboard and saving feed == should set to clear board after 2 weeks? SH """
        # Add the post to the feed
        # post_text = post_entry.get()
        # post_feed.insert(END, post_text)
        # post_entry.delete(0, END) # clear the entry field after posting
        
        pass 
        
        
        
        

In [74]:
"""
    Window to appear when the user has created an account successfully or logged in successfully
    
"""

class UserProfile: # added class for user profiles once logged in or account is created --  SH 
    def __init__(self, firstname, lastname): # added argument for user profiles firstname and lastname to appear on welcome label
        # Initialize top level window
        self.profile = Toplevel()        
        self.profile.title(" Resource Treatment Center || Account")
        self.profile.geometry('800x800')
        self.center_window(800, 800)      # JS centers window
        self.profile.resizable(True, True)
        self.profile.config(background="light grey")
        
        # Create a Canvas widget to hold background image and label/button widgets
        self.canvas = Canvas(self.profile, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="fist-bump.png") # adjust image to display neutral colors
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        
        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)

	    # define Colors
        self.label_font_color = "navy"

	    # Label background color
        self.label_bg = "white"

	    # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg=("navy")
        self.button_fg=("light gray")

	    # Diss Board Positng background color
        self.post_background = "light grey"
        
        # Title for the Profile window
        self.welcome_label = Label(self.profile, text=f"Welcome, {firstname} {lastname} to RTC Support Network!", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg) # SH added variables to display name on profile
        # place label on canvas
        self.canvas.create_window(400, 50, window=self.welcome_label)
        # create detailed message label
        self.detailMessage = Label(self.profile, text="RTC's Network Application where communication is always welcome! \n Enjoy chatting in the open discussion board or private message a friend or counselor at any moment!", fg=self.label_font_color, font=self.entry_font, bg=self.label_bg)
        # place label on canvas
        self.canvas.create_window(400, 150, window=self.detailMessage)
        
        
        # Peer Chat Page button
        self.peerChat_button = Button(self.profile, text="Private Message Peers", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.OpenPeerChat)
        self.canvas.create_window(400, 250, window=self.peerChat_button)
        
        # Counselor Chat Page button
        self.counselorChat_button = Button(self.profile, text="Private Message a Counselor", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.OpenCounselorChat)
        self.canvas.create_window(400, 350, window=self.counselorChat_button)
        
        # Diss Board Page button
        self.dissBoard_button = Button(self.profile, text="Open Discussion Board", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.OpenDissBoard)
        self.canvas.create_window(400, 450, window=self.dissBoard_button)
        
    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.profile.winfo_screenwidth()
        screen_height = self.profile.winfo_screenheight()
                # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.profile.geometry(f"{width}x{height}+{x}+{y}") 
        
        
    # placeholder methods for the commands to open more windows
    def OpenPeerChat(self):
        """
            Open Peer Chat Window when button is clicked via profile
        """
        PeerChat()
        
        
    def OpenCounselorChat(self):
        """
            Open the Counselor Chat window when button is clicked via profile 
        """
        CounselorChat()
        
        
    def OpenDissBoard(self):
        """
            Open the Discussion Board window when button is clicked via profile 
        """
        DissBoard()


In [75]:
"""
Login Window  
Created by Sammi Harper
This is the login window of the application to allow users to log in and access the account features. 

"""


class LoginWindow():
    def __init__(self):
        # Initialize top level window
        self.login = Toplevel()
        self.login.title("Resource Treatment Center || LOGIN")
        self.login.geometry('500x500')
        self.center_window(500, 500)
        self.login.resizable(True, True)
        
        # Create a Canvas widget to hold background image and label/button widgets     
        self.canvas = Canvas(self.login, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="login_fist_bump.PNG") # adjust image size to fit
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
    

        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)

        # Define Colors
        self.label_font_color = "navy"
        self.label_bg = "white"
        self.entry_bg = "light grey"

        # Button specs
        self.button_bg = "navy"
        self.button_fg = "light gray"

        # Title for the login window
        self.title_label = Label(self.login, text="Login to your Account", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(250, 75, window=self.title_label)

        # Username label and entry
        self.username_label = Label(self.login, text="Username:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 125, window=self.username_label)
        self.username_entry = Entry(self.login, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 125, window=self.username_entry)

        # Password label and entry
        self.password_label = Label(self.login, text="Password:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 175, window=self.password_label)
        self.password_entry = Entry(self.login, font=self.entry_font, bg=self.entry_bg, show="*")
        self.canvas.create_window(250, 175, window=self.password_entry)

        # Login button
        self.login_button = Button(self.login, text="Login", font=self.entry_font, bg=self.button_bg, fg=self.button_fg, command=self.validate_login)
        self.canvas.create_window(250, 225, window=self.login_button)   

    def center_window(self, width, height): # JS centers login window in center of screen
        screen_width = self.login.winfo_screenwidth()
        screen_height = self.login.winfo_screenheight()
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.login.geometry(f"{width}x{height}+{x}+{y}")
        
    def validate_login(self):
        """
        Validate the login credentials and open the User Profile if valid. Otherwise, display an error message.  
        """
        
        # Get username and password from the entries
        username = self.username_entry.get()
        password = self.password_entry.get()

        # Check if username and password exists
        user_info = self.check_credentials(username, password)
        if user_info:
            firstname, lastname = user_info
            messagebox.showinfo("Login Success", f"Welcome {firstname} {lastname}!")
            self.login.destroy()
            UserProfile(firstname, lastname)  # Open the account window on successful login with user's name
            
        else:  # display error if credentials are not entered correctly
            messagebox.showerror("Login Failed", "Invalid username or password")

    def check_credentials(self, username, password):
        """
        Validate the login values against the database. Returns the first name and lastname if valid, otherwise None.
        """
        try:
            # Connect to the database
            conn = sqlite3.connect('support_network.db')  # Updated to match your CreateAccount class database
            cursor = conn.cursor()

            # Check if the username and password exist
            cursor.execute("SELECT p_first_name, p_last_name FROM patient_users WHERE p_username=? AND p_password=?", (username, password))
            result = cursor.fetchone()

            conn.close()

            if result:
                return result  # Returns firstname and lastname
            else:
                return None

        except sqlite3.Error as e:
            messagebox.showerror("Database Error", f"An error occurred: {e}")
            return None


In [ ]:
"""
Create Account Window  
Created by Sammi Harper
This is the window fpr users to create an account to access the account features. 

"""


class CreateAccount():
    def __init__(self):
        # Initialize top level window
        self.CreateAccount = Toplevel()
        self.CreateAccount.title(" Resource Treatment Center || CREATE Account")
        self.CreateAccount.geometry('500x500')
        self.center_window(500, 500)      # JS centers window
        self.CreateAccount.resizable(True, True)
        self.CreateAccount.config(background="light grey")
        
        self.canvas = Canvas(self.CreateAccount, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="login_fist_bump.PNG") # adjust image size to fit
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        
        # Define Fonts
        self.title_label_font = ("Helvetica", 22)
        self.label_font = ("Helvetica", 14)
        self.entry_font = ("Helvetica", 12)
        
        # define Colors
        self.label_font_color = "navy"
        
        # Label background color
        self.label_bg = "white" # SH changed label background color to white 
        self.entry_bg = "light grey"  # sh added entry background color

        # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg=("navy")
        self.button_fg=("light gray")
        
        
        # title for the CreateAccount window
        self.title_label = Label(self.CreateAccount, text="Create your Account", font=self.title_label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(250, 75, window=self.title_label)
        
        # Username label and entry
        self.username_label = Label(self.CreateAccount, text="Username:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 125, window=self.username_label)
        self.username_entry = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 125, window=self.username_entry) 
        
        # Password label and entry
        self.password_label = Label(self.CreateAccount, text="Password:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 160, window=self.password_label)
        self.password_entry = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg, show="*")
        self.canvas.create_window(250, 160, window=self.password_entry)
        
        # First name label and entry
        self.first_name = Label(self.CreateAccount, text="First Name:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 195, window=self.first_name)
        self.first_name = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 195, window=self.first_name)
        
        # Last name label and entry
        self.last_name = Label(self.CreateAccount, text="Last Name:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 230, window=self.last_name)
        self.last_name = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 230, window=self.last_name)
        
        # Email label and entry
        self.email = Label(self.CreateAccount, text="Email Address:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 265, window=self.email)
        self.email = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 265, window=self.email)
        
        # Phone Number label and entry
        self.phone_number = Label(self.CreateAccount, text="Phone Number:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 300, window=self.phone_number)
        self.phone_number = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 300, window=self.phone_number)
        
        #Counselor Name label and entry
        self.Counselor = Label(self.CreateAccount, text="Counselor's ID:", font=self.label_font, fg=self.label_font_color, bg=self.label_bg)
        self.canvas.create_window(75, 335, window=self.Counselor)
        self.Counselor = Entry(self.CreateAccount, font=self.entry_font, bg=self.entry_bg)
        self.canvas.create_window(250, 335, window=self.Counselor)
        
        # Create Account button
        self.CreateAccount = Button(self.CreateAccount, text="Create Account", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=self.createAcc) # JS create account button linked to createAcc to provide message box
        self.canvas.create_window(250, 380, window=self.CreateAccount)
        
        
        # initialize database
        self.init_support_net_db()
        

    def init_support_net_db(self):
        """
        Initializes the SQLite database and creates necessary tables for patients and counselors.
        Ensures that the database connection is properly closed to prevent locking issues.
        """
        
        # SH had to add try and accept for code or DB locked and application broke
        try: 
            conn = sqlite3.connect('support_network.db')
            cursor = conn.cursor()
            patient_user = """CREATE TABLE IF NOT EXISTS patient_users (
                p_id INTEGER PRIMARY KEY NOT NULL,
                p_username TEXT UNIQUE NOT NULL,
                p_password TEXT NOT NULL,
                p_first_name TEXT NOT NULL,
                p_last_name TEXT NOT NULL,
                p_email TEXT UNIQUE NOT NULL,
                p_phone_number TEXT UNIQUE NOT NULL,
                c_id INTEGER NOT NULL,
                FOREIGN KEY(c_id) REFERENCES counselor_users(c_id))"""
                
                
            cursor.execute(patient_user) # SH changed the counselor last name to counselor id to be a foreign key referenceing the counselor table
            conn.commit()
        
                    
            # TODO: James --> add table to for counselors #JS 2/22 unsure if this is correct --> SH these are column names for the DB - they should reference the counselor details not the patient table. 
            # changed variables to have a c in front rather than a p -- SH
            counselor_user = """CREATE TABLE IF NOT EXISTS counselor_users (
                C_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                c_username TEXT UNIQUE NOT NULL,           
                c_password TEXT NOT NULL,
                c_first_name TEXT NOT NULL,
                c_last_name TEXT NOT NULL,
                c_email TEXT UNIQUE NOT NULL,
                c_phone_number TEXT UNIQUE NOT NULL)"""
            # TODO: James --> add cursor.execute command to add each counselor in DB (maybe like 5ish???)

            # Create the counselor_users table
            cursor.execute(counselor_user)

            # Insert counselor(s) only if he/she does not already exist
            cursor.execute("SELECT * FROM counselor_users WHERE c_id = 1")
            if cursor.fetchone() is None:
                cursor.execute("INSERT INTO counselor_users (c_id, c_username, c_password, c_first_name, c_last_name, c_email, c_phone_number) VALUES (1, 'TestCounselor', 'Test', 'John', 'Doe', 'jd@test.com', '1233211122')")
            conn.commit()
        except sqlite3.OperationalError as e:
            messagebox.showerror("Database Error", f"An error occurred: {e}")
        finally:
            conn.close()  # Ensure the connection is closed
        
    
    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.CreateAccount.winfo_screenwidth()
        screen_height = self.CreateAccount.winfo_screenheight()

        # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.CreateAccount.geometry(f"{width}x{height}+{x}+{y}")  
        
    def validate_input(self, p_username, p_password, p_first_name, p_last_name, p_email, p_phone_number, c_id):
        """
        This method validates the user input: username, password, first name, last name, email, phone number, and counselor's ID.
        Returns: True if all inputs are valid, False otherwise (False will display a message).
        Regex input validation site
        https://www.contentstack.com/docs/developers/create-content-types/validation-regex 
        """
        
        if not all([p_username, p_password, p_first_name, p_last_name, p_email, p_phone_number, c_id]):
            messagebox.showerror("Error", "All fields are required!")
            return False
        
        # Password validation: Must be at least 8 characters long and contain at least one letter and one digit
        password_regex = (r'^(?=.*[A-Za-z])(?=.*\d)[A-Za-z\d]{8,}$')
        if not re.match(password_regex, p_password):
            messagebox.showerror("Error", "Password should be at least 8 characters long, and contain at least one letter and one number!")
            return False
        
        # Name validation: Only letters and spaces, between 2 and 30 characters
        name_regex = (r'^[A-Za-z\s]{2,30}$')
        if not re.match(name_regex, p_first_name) or not re.match(name_regex, p_last_name):
            messagebox.showerror("Error", "First and last names should only contain letters and spaces, and be between 2 and 30 characters.")
            return False
        
        # Email validation: Basic email format check
        email_regex = (r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
        if not re.match(email_regex, p_email):
            messagebox.showerror("Error", "Please enter a valid email address.")
            return False
        
        # Phone number validation: Only digits, and can be 10-15 characters long (optional + at the start)
        phone_regex = (r'^\+?[0-9]{10,15}$')
        if not re.match(phone_regex, p_phone_number):
            messagebox.showerror("Error", "Please enter a valid phone number (optional +, followed by 10-15 digits).")
            return False
        
        # Counselor ID validation: Ensure it's numeric
        if not c_id.isdigit():  
            messagebox.showerror("Error", "Counselor's ID should be a number!")
            return False
        
        try:
            conn = sqlite3.connect("support_network.db")
            cursor = conn.cursor()

            # Check for existing username, email, and phone number
            cursor.execute("SELECT * FROM patient_users WHERE p_username = ? OR p_email = ? OR p_phone_number = ?", (p_username, p_email, p_phone_number))
            if cursor.fetchone():
                messagebox.showerror("Error", "Username, email, or phone number already exists!")
                conn.close()
                return False

            # Check if counselor ID exists in counselor_user table
            cursor.execute("SELECT * FROM counselor_users WHERE c_id = ?", (c_id,))
            if not cursor.fetchone():
                messagebox.showerror("Error", "Counselor's ID not found!")
                conn.close()
                return False

            conn.close()
            
        except sqlite3.Error as e:  # Handle any SQLite database errors
            messagebox.showerror("Database Error", f"An error occurred: {e}")
            return False
        
        return True



    def createAcc(self):
        """
        Collects user input, validates the data, and inserts a new patient record into the database.
        Ensures proper error handling to prevent database integrity issues.
        """
        # .get command must be listed as the Entry Box name from creating the input box 
        username = self.username_entry.get()
        firstname = self.first_name.get()
        lastname = self.last_name.get()
        email = self.email.get()
        phone = self.phone_number.get()
        password = self.password_entry.get()
        c_id = self.Counselor.get()

        # Validate the input fields and create the account if valid
        if not self.validate_input(username, firstname, lastname, email, phone, password, c_id):
            return # Stop execution if validation fails
        
        try:
            conn = sqlite3.connect("support_network.db")
            cursor = conn.cursor()
            # Insert the new patient record into the patient_users table
            cursor.execute('''INSERT INTO patient_users (p_username, p_first_name, p_last_name, p_email, p_phone_number, p_password, c_id)
                              VALUES (?, ?, ?, ?, ?, ?, ?)''', (username, firstname, lastname, email, phone, password, c_id))
            conn.commit()
            conn.close()
            messagebox.showinfo("Success", "Account created successfully!")
            self.CreateAccount.destroy()
        # Push error message if validation fails or other DB error occurs
        except (sqlite3.IntegrityError, sqlite3.Error) as e:
            messagebox.showerror("Error", f"An error occurred: {e}")



In [ ]:
"""
Main Window GUI 
Created by Sammi Harper
This is the main window of the application to allow users to log in or create an account. 

"""

class MainWindow():
    def __init__(self, master_window):
        # Setting and creating window to be resizable by user
        self.master = master_window
        self.master.title(" Resource Treatment Center ")
        self.master.geometry('800x800')
        self.center_window(800, 800)      # JS centers window
        self.master.resizable(True, True)

        self.canvas = Canvas(self.master, width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        
        # Load and display the image as background
        self.account_banner = PhotoImage(file="fist-bump.png") # adjust image size to fit
        self.canvas.create_image(0, 0, image=self.account_banner, anchor="nw")
        # Application font
        self.label_font = ("Helvetica", 25)

        # Sub-label font sizes
        self.sub_label_font = "Helvetica", 16

        # Button specs
        self.button_font = Font(family="Helvetica", size=12, weight="bold")
        self.button_bg = "navy"
        self.button_fg = "light gray"

        # Top label font size and color
        self.top_font = Font(family="Helvetica", size=35, weight="bold")
        # Color for top label font
        self.color_font = "Navy"
        self.bg = "white"


        # Create a welcome label with title and subtitle on the top of the window
        title = Label(self.master, text="Support Network Application", fg=self.color_font, bg=self.bg,
                    font=self.top_font)
        # Position the label on the top of the window using place()
        title.place(relx=0.5, rely=0.1, anchor="center")

        subTtitle = Label(self.master, text="Please log in or create a new account", fg=self.color_font, font=self.sub_label_font, bg=self.bg)
        # Position the sub-title below the title using place()
        subTtitle.place(relx=0.5, rely=0.2, anchor="center")

        # CREATE LOGIN BUTTON
        login_button = tk.Button(self.master, text="Login to Account", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=LoginWindow)
        # Position the login button using place()
        login_button.place(relx=0.35, rely=0.5, anchor="center")

        # CREATE CREATE ACCOUNT BUTTON
        create_account_button = tk.Button(self.master, text="Create an Account", font=self.button_font, bg=self.button_bg, fg=self.button_fg, command=CreateAccount)
        # Position the create account button using place()
        create_account_button.place(relx=0.65, rely=0.5, anchor="center")

    def center_window(self, width, height):  # JS adding so when create account window opened, finds center of screen
        # Get screen width and height
        screen_width = self.master.winfo_screenwidth()
        screen_height = self.master.winfo_screenheight()

        # Calculate x and y coordinates for the Tk window
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)

        # Set the geometry
        self.master.geometry(f"{width}x{height}+{x}+{y}")

    def login(self):
        print("Login button clicked!")

    def create_account(self):
        print("Create Account button clicked!")

def main():
    root = tk.Tk()
    window = MainWindow(root)
    root.mainloop()

if __name__ == '__main__':
    main()
